## PART 2

# Problem 3

In [1]:
using CSV, DataFrames
filepath = "interpolated_height.csv"
df = CSV.read(filepath, DataFrame)	
H = round.(df[:, 2], digits=0)

317-element Vector{Float64}:
   0.0
   2.0
   5.0
   7.0
   9.0
  12.0
  14.0
  17.0
  19.0
  20.0
   ⋮
  54.0
  28.0
  10.0
  22.0
  34.0
  47.0
  27.0
   0.0
 -27.0

In [4]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[k - abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )

    @objective(myModel, Min, sum(x[j] for j=1:h) )

    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -seconds 20 -solve -quit (default strategy 1)
seconds was changed from 1e+100 to 20
Continuous objective value is 48.9519 - 0.00 seconds
Cgl0004I processed model has 316 rows, 317 columns (317 integer (317 of which binary)) and 1576 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 273 integers unsatisfied sum - 71.0119
Cbc0038I Pass   1: suminf.   66.84121 (261) obj. 78.661 iterations 54
Cbc0038I Pass   2: suminf.   52.10023 (203) obj. 80.751 iterations 93
Cbc0038I Pass   3: suminf.   37.19367 (148) obj. 86.992 iterations 109
Cbc0038I Pass   4: suminf.   30.17432 (123) obj. 89.9798 iterations 56
Cbc0038I Pass   5: suminf.   26.05603 (117) obj. 92.3961 iterations 40
Cbc0038I Pass   6: suminf.   24.57564 (112) obj. 93.5756 iterations 21
Cbc0038I Pass   7: suminf.    6.04388 (45) obj. 105.875 iterations 75
Cbc0038I Pass   8: suminf.    5.19911 (42) obj

# Problem 3

In [5]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[k - abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )

    @objective(myModel, Min, (abs(R[j]-H[j]-10) for j=1:h) )

    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

ErrorException: The objective function `Base.Generator{UnitRange{Int64}, var"#37#42"{Vector{Float64}, Vector{VariableRef}}}(var"#37#42"{Vector{Float64}, Vector{VariableRef}}([0.0, 2.0, 5.0, 7.0, 9.0, 12.0, 14.0, 17.0, 19.0, 20.0, 21.0, 21.0, 22.0, 22.0, 23.0, 23.0, 23.0, 22.0, 22.0, 23.0, 23.0, 24.0, 27.0, 29.0, 31.0, 32.0, 32.0, 33.0, 34.0, 34.0, 35.0, 37.0, 37.0, 37.0, 37.0, 38.0, 39.0, 40.0, 41.0, 43.0, 45.0, 47.0, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 58.0, 57.0, 59.0, 66.0, 74.0, 82.0, 87.0, 93.0, 98.0, 101.0, 102.0, 104.0, 103.0, 103.0, 102.0, 102.0, 103.0, 104.0, 104.0, 105.0, 105.0, 107.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 116.0, 118.0, 119.0, 122.0, 125.0, 128.0, 128.0, 128.0, 128.0, 129.0, 129.0, 130.0, 131.0, 132.0, 132.0, 133.0, 133.0, 135.0, 136.0, 137.0, 138.0, 137.0, 136.0, 137.0, 137.0, 138.0, 137.0, 136.0, 136.0, 136.0, 136.0, 137.0, 138.0, 138.0, 139.0, 139.0, 140.0, 140.0, 140.0, 139.0, 139.0, 139.0, 139.0, 139.0, 140.0, 140.0, 141.0, 142.0, 143.0, 143.0, 142.0